In [1]:
import numpy as np
import glob
import pickle
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from data_formatting import * 
import data_formatting
from imp import reload

reload(data_formatting)

In [35]:
enigma=pd.read_csv('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/cortical_atrophy/data/Behaviour_Information_ALL_April7_2022.csv', header=0)
idx_names=enigma[pd.isna(enigma['NORMED_MOTOR_CHRONIC'])].index
enigma.drop(idx_names, inplace=True)
enigma=enigma[enigma['SEX']!=3.0]
enigma=enigma[enigma['LESIONED_HEMISPHERE']!=0]
enigma=enigma[enigma['AGE']!='nan']
enigma=enigma[enigma['AGE']!='NaN']

enigma=enigma[enigma['NORMED_MOTOR']!='nan']

enigma=enigma.drop(columns=[ 'CHRONIC', 'ACUTE', 'NORMED_MOTOR_ACUTE', 'NORMED_MOTOR_CHRONIC', 'FUGL_MEYER_UE_NORM', 'isblank_sex', 'isblank_age', 'isblank_days_post_stroke', 'isblank_lesionedhem', 'isblank_fuglmeyer', 'isblank_chronicity', 'is present_chronicity'])
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 0, 'LESIONED_HEMISPHERE'] = 'undefined'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 1, 'LESIONED_HEMISPHERE'] = 'left cerebrum'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 2, 'LESIONED_HEMISPHERE'] = 'right cerebrum'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 3, 'LESIONED_HEMISPHERE'] = 'bilateral'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 4, 'LESIONED_HEMISPHERE'] = 'brainstem'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 5, 'LESIONED_HEMISPHERE'] = 'cerebellum'
enigma.loc[enigma['LESIONED_HEMISPHERE'] == 6, 'LESIONED_HEMISPHERE'] = 'cerebrum + brainstem/cereb.'

enigma.loc[enigma['SEX'] == 1, 'SEX'] = 'female'
enigma.loc[enigma['SEX'] == 2, 'SEX'] = 'male'

enigma=enigma[enigma['CHRONICITY']==180]
enigma=enigma.dropna()

In [4]:


## all settings:
y_var = 'normed_motor_scores'
subset = 'chronic'
model_tested = ['ridge']
verbose = True
covariates=['SEX', 'AGE', 'DAYS_POST_STROKE', 'LESIONED_HEMISPHERE']

lesionload_type = 'M1' # options: 
nperms=5

save_models=1
lesionload_types = ['none','M1','all']
lesionload_type = 'none'
ensembles = ['none', 'demog']
atlas = [ 'shen268']
chaco_types = ['chacovol']
chaco_type = 'chacovol'
crossval_types =['2', '4']
null=-1

LESIONMASK_PATH = os.path.join("//Users/emilyolafson/GIT/ENIGMA/data/lesionmasks/all_lesionmasks_2009tov6_usingSym")  # Set this path accordingly
CSV_PATH = os.path.join("~/GIT/ENIGMA/data/Behaviour_Information_ALL_April7_2022_sorted_CSTll.csv")  # Set this path accordingly

[X, Y, C, lesion_load, site] = create_data_set(CSV_PATH,atlas,covariates, verbose, y_var, chaco_type, subset,1,ll= lesionload_type)


Removing subjects who do not have motor scores.
---- Original N = 1068
---- New N = 1020


Removing subjects that do not have any of: ['SEX', 'AGE', 'DAYS_POST_STROKE', 'LESIONED_HEMISPHERE']
---- Original N = 1020
---- New N = 777




Removing subjects that are not chronic stroke subjects
---- Original N = 777
---- New N = 462
Removing subjects who do not have lesion masks.
---- Original N = 462
---- New N = 0


Loading data for atlas: ['shen268'], ChaCo scores: chacovol, subset: chronic


UnboundLocalError: local variable 'X' referenced before assignment

In [11]:
enigma.describe()

,AGE,DAYS_POST_STROKE,NORMED_MOTOR
count,624.000000,519.000000,625.000000
mean,60.679225,1351.286609,0.655527
std,12.287929,1370.318946,0.282784
min,23.000000,181.000000,0.000000
25%,53.000000,461.000000,0.424242
50%,62.000000,871.000000,0.712121
75%,69.000000,1659.000000,0.924242
max,93.000000,10806.000000,1.000000


In [27]:
np.unique(enigma['SITE'])

sumary= []
names = []
i=0
for site in np.unique(enigma['SITE']):
    sumz = np.sum(enigma['SITE']==site)
    sumary.append(sumz)
    names.append(site)

data = {'Site': names, 'Sums':sumary}
table = pd.DataFrame(data)
print(table.to_latex(index=True)) 

\begin{tabular}{llr}
\toprule
{} &  Site &  Sums \\
\midrule
0  &  r001 &    39 \\
1  &  r002 &    12 \\
2  &  r003 &    15 \\
3  &  r004 &    19 \\
4  &  r005 &    27 \\
5  &  r009 &    57 \\
6  &  r010 &    26 \\
7  &  r011 &    28 \\
8  &  r015 &    15 \\
9  &  r017 &    14 \\
10 &  r018 &    11 \\
11 &  r021 &    12 \\
12 &  r022 &    14 \\
13 &  r023 &    14 \\
14 &  r024 &    21 \\
15 &  r025 &    16 \\
16 &  r027 &    28 \\
17 &  r028 &    21 \\
18 &  r029 &     5 \\
19 &  r031 &     1 \\
20 &  r033 &     5 \\
21 &  r034 &    15 \\
22 &  r035 &    15 \\
23 &  r038 &    18 \\
24 &  r040 &    14 \\
25 &  r042 &    22 \\
26 &  r044 &     4 \\
27 &  r045 &     4 \\
28 &  r046 &    12 \\
29 &  r047 &    44 \\
30 &  r048 &    43 \\
31 &  r052 &    32 \\
32 &  r053 &     2 \\
\bottomrule
\end{tabular}



In [188]:
plt.figure()
sns.color_palette("cubehelix", as_cmap=True)
pal = sns.cubehelix_palette(2, rot=-.25, light=.7)

gris = sns.pairplot(enigma,vars=['NORMED_MOTOR', 'DAYS_POST_STROKE', 'AGE'], hue='SEX',plot_kws={"s": 10},palette=pal)
for ax in gris.axes.flat[:2]:
    ax.tick_params(axis='x', labelrotation=90)
    
    
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/pairplot.pdf')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/pairplot.png')

In [209]:
enigma.corr()
calculate_pvalues(enigma) 


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [213]:
from scipy.stats import pearsonr, spearmanr
rho = enigma.corr(method = 'spearman')

pval = enigma.corr(method=lambda x, y: spearmanr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
rho.round(2).astype(str) + p

,AGE,DAYS_POST_STROKE,NORMED_MOTOR
AGE,1.0***,0.06,0.17***
DAYS_POST_STROKE,0.06,1.0***,-0.14***
NORMED_MOTOR,0.17***,-0.14***,1.0***


In [184]:
plt.figure()
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

rs = np.random.RandomState(1979)
x = rs.randn(628)
g = sns.FacetGrid(enigma, row="LESIONED_HEMISPHERE", hue="LESIONED_HEMISPHERE", aspect=7, height=1, palette=pal)

# Draw the densities in a few steps
g.map(sns.histplot, "NORMED_MOTOR", stat='count', bins=50)
# bw_adjust=.1, clip_on=True,fill=True, alpha=0.4, linewidth=1.5)
#g.map(sns.kdeplot, "NORMED_MOTOR", clip_on=False, color="w", lw=2, bw_adjust=.5)

# passing color=None to refline() uses the hue mapping
g.map(plt.axhline, y=0, linewidth=1, linestyle="-", color=None, clip_on=False)

# Define and use a simple function to label the plot in axes coordinates
def label(x, color, label):
    ax = plt.gca()
    ax.text(-0.2, .2, '',color=color,
            ha="center", va="center", transform=ax.transAxes)
    ax.set_title(label, color=color)
    ax.yaxis.set_tick_params(labelsize = 10, color='blue')

g.map(label, "NORMED_MOTOR")

# Set the subplots to overlap
#g.fig.subplots_adjust(hspace=-.1)

# Remove axes details that don't play well with overlap
#g.set_titles("")
g.set(xlim=(-0.1, 1),ylim=(-0.1, 10), ylabel='Count')
g.despine(bottom=True, left=True)

plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/LESIONED.pdf')
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/LESIONED.png')

In [182]:
label 

<function __main__.label(x, color, label)>

In [92]:

gris = sns.ridgeplot(enigma,x_vars=['NORMED_MOTOR'], hue='SEX', y_vars = 'LESIONED_HEMISPHERE',plot_kws={"s": 5}, kind='hist')
    


AttributeError: module 'seaborn' has no attribute 'ridgeplot'

In [55]:
enigma.dtypes

BIDS_ID                 object
SES                     object
SITE                    object
AGE                    float64
SEX                    float64
DAYS_POST_STROKE       float64
LESIONED_HEMISPHERE     object
NORMED_MOTOR           float64
dtype: object

In [44]:
sitesize=[]
for i in np.unique(enigma['SITE']):
    sitesize.append(np.sum(enigma['SITE'] ==i))
np.min(sitesize)
np.median(sitesize)

15.0

In [ ]:
print(np.sum(enigma['SEX']==2))
np.sum(enigma['SEX']==1)

345


185

In [ ]:
plt.figure()

sns.histplot(enigma['NORMED_MOTOR'])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/normed_motor.png')

plt.figure()
sns.histplot(enigma['AGE'])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/age.png')

plt.figure()
sns.histplot(enigma['DAYS_POST_STROKE'])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/timesincestroke.png')


In [ ]:
plt.figure()
sns.histplot(x='SEX', hue = 'SEX',data=enigma)


plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/question.png')


In [ ]:
plt.figure()
sns.histplot(x='AGE', data=enigma[enigma['SEX']==1])
plt.ylim([0,70])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/female.png')


plt.figure()
sns.histplot(x='AGE', data=enigma[enigma['SEX']==2])
plt.ylim([0,70])
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/male.png')



In [ ]:

lesvols = []
for sub in enigma['BIDS_ID']:
    lesionvol = np.loadtxt('/Users/emilyolafson/GIT/ENIGMA/data/lesionvol/{}_ses-1.lesionvol.txt'.format(sub))
    lesionvol = lesionvol[0]
    lesvols.append(lesionvol)

In [ ]:

plt.figure()
sns.histplot(x=np.log(elesvols)
plt.savefig('/Users/emilyolafson/GIT/ENIGMA/enigma_disconnections/figures/lesvols.png')



<ipython-input-97-5dc7ca40e1d7>:2: RuntimeWarning: divide by zero encountered in log
  sns.histplot(x=np.log(lesvols))


ValueError: autodetected range of [-inf, 12.866841252026097] is not finite